In [38]:
import pandas as pd
import scipy.stats as ss
import numpy as np

df = pd.read_stata('hsbdemo.dta')
df = df.ix[:,1:]
df = df.ix[:,:-1]
df[199:]

,female,ses,schtyp,prog,read,write,math,science,socst,honors,awards
199,male,middle,public,academic,73.0,62.0,73.0,69.0,66.0,enrolled,3.0


In [28]:
y_data = pd.get_dummies(df.ix[:,"honors"]).as_matrix()
x_data_df = df.drop(["honors"],axis=1)

x_data_df.insert(0, "bias", 1)

In [29]:
x_data = pd.get_dummies(x_data_df).as_matrix()

import scipy.stats as ss
x_data[:,1:] = np.array(ss.zscore(x_data[:,1:]))

In [30]:
import tensorflow as tf

X = tf.placeholder("float", [None, len(x_data[0])])
Y = tf.placeholder("float", [None, len(y_data[0])])

W = tf.Variable(tf.zeros([len(x_data[0]), len(y_data[0])]))

hypothesis = tf.nn.softmax(tf.matmul(X,W))
learning_rate = 0.001

cost = tf.reduce_mean(-1 * tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)
    
for i in range(40001):
    sess.run(optimizer, feed_dict={X: x_data, Y:y_data})
    if i % 20000 == 0:
        print ("%d 's iteration" % i)
        print (sess.run(cost, feed_dict={X: x_data, Y:y_data}))

0 's iteration
0.692156
20000 's iteration
0.119084


In [31]:
import numpy as np

total_value = 0.0
for count in range(100):
    test_index = np.random.choice(len(x_data), len(x_data) * 0.3)
    number_of_test = len(x_data) * 0.3
    hypotehsis_value = sess.run(hypothesis, feed_dict={X:x_data[test_index]})
    hypotehsis_result = sess.run( tf.arg_max(hypotehsis_value,1))

    test_result = [np.argmax(predict) ==  np.argmax(original) 
                   for predict, original in zip(hypotehsis_value, y_data[test_index])]


    total_value += np.sum(test_result) / number_of_test

print( total_value / 100)

/Users/sungchulchoi/miniconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.996833333333
